In [ ]:
!pip install sentence_transformers

In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch.nn.functional as F
from sentence_transformers import models as sentence_models
import pandas as pd

In [2]:
nsmarco = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')


/home/benk/anaconda3/envs/DM_ENV/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:

df = pd.read_csv('movies_dataset.csv')
df['Genre'] = df['Genre'].replace('unknown', '')
df['Director'] = df['Director'].replace('Unknown', '')
df['search'] = df['Title'] + ', ' + df['Release Year'].astype(str) + ', ' + \
               df['Genre'] + ', ' + df['Origin/Ethnicity'] + ', ' + \
               df['Director'] + ': ' + df['Plot']

data_plots = df['search'].tolist()

In [14]:
df.to_csv('movies_dataset_saved.csv', index=False)


In [5]:
import pickle

# Save the model
with open("nsmarco_model.pkl", "wb") as f:
    pickle.dump(nsmarco, f)

In [6]:
document_embeddings = nsmarco.encode(data_plots)


In [7]:


# Save the document embeddings
with open("document_embeddings.pkl", "wb") as f:
    pickle.dump(document_embeddings, f)


In [8]:
def semantic_search(query, data=data_plots, movie_data=df):
    query_embedding = nsmarco.encode([query])[0]

    similarities = cosine_similarity([query_embedding], document_embeddings)[0]  # type: ignore

    sorted_results = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)

    search_results = [(str(idx+1), movie_data.iloc[idx]['Title'], similarity) for idx, similarity in sorted_results]
    return search_results


In [9]:
import pickle

# Load the model
with open("nsmarco_model.pkl", "rb") as f:
    nsmarco = pickle.load(f)

# Load the document embeddings
with open("document_embeddings.pkl", "rb") as f:
    document_embeddings = pickle.load(f)

In [12]:
# query = 'decaprio in a big boat with an older lady going from uk to usa'
query = 'decaprio'

r1 = semantic_search(query)
print(r1)

[('9586', 'The Children of Sanchez', 0.45539552), ('318', 'A Double-Dyed Deceiver', 0.45302308), ('23841', 'The Trial', 0.4501475), ('33915', 'Genocidal Organ', 0.44763476), ('16255', 'Do-Deca-Pentathlon, TheThe Do-Deca-Pentathlon', 0.4471736), ('19078', 'The Fake', 0.44359028), ('18870', 'Give Us This Day', 0.4419381), ('22225', 'The Bang Bang Club', 0.4407081), ('13142', 'Strategic Command', 0.4375144), ('17818', 'Blessed', 0.43642032), ('3', 'The Martyred Presidents', 0.43588737), ('28472', 'John Paul Vaathil Thurakkunnu', 0.4350256), ('1106', 'The Drums of Jeopardy', 0.43381017), ('16724', 'Open Grave', 0.43269408), ('7373', 'Let No Man Write My Epitaph', 0.4321178), ('10356', 'Dungeonmaster, The', 0.4306671), ('11481', 'Narrow Margin', 0.42879295), ('772', 'The Case of Lena Smith', 0.42736855), ('17527', 'The Return of Captain Invincible', 0.42709538), ('18653', 'San Demetrio London', 0.42685622), ('1874', 'The Devil is a Woman', 0.42539826), ('13841', 'Escape from Hell', 0.425338